In [ ]:
import csv
import json
from shapely.geometry import shape, Point

# Load GeoJSON data for Haryana
with open('/content/HARYANA_STATE.geojson', 'r') as geojson_file:
    haryana_data = json.load(geojson_file)
    haryana_boundary = shape(haryana_data['features'][0]['geometry'])

# Read the CSV file
csv_file_path = '/content/ind_pak_relative_wealth_index (1).csv'
filtered_rows = []

with open(csv_file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Extract latitude and longitude from the CSV row
        latitude = float(row['latitude'])
        longitude = float(row['longitude'])

        # Check if the latitude and longitude fall inside Haryana
        point = Point(longitude, latitude)
        if haryana_boundary.contains(point):
            filtered_rows.append(row)

# Write the filtered rows to a new CSV file in /kaggle/working/
output_csv_path = '/content/filtered_data.csv'  # Changed output path
fieldnames = reader.fieldnames
with open(output_csv_path, 'w', newline='') as outfile:
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(filtered_rows)

print('Filtered data saved to', output_csv_path)

Filtered data saved to /content/filtered_data.csv


First the csv file needs to be filtered so that we get the data of Haryana only. This is done using the above code block. Then, latitude and longitude of the filtered csv file are used along with the latitude and longitude of our csv file to get the closest RWI value for each patch. This is saved in the CSV file "csv file with labels.csv".

In [ ]:
  import pandas as pd

  # Load the CSV files into pandas DataFrames
  y_data = pd.read_csv('/content/filtered_data.csv')
  x_data = pd.read_csv('/content/deep + atm.csv')

  # Function to calculate the distance between two coordinates
  def calculate_distance(lat1, lon1, lat2, lon2):
      coord1 = (lat1, lon1)
      coord2 = (lat2, lon2)
      dist=(lat1-lat2)**2 + (lon1-lon2)**2
      return dist

  # Function to find the closest NTL for each image
  def find_closest_rwi(row):
      min_distance = float('inf')
      closest_rwi = None

      for idx, y_row in y_data.iterrows():
          distance = calculate_distance(row['Latitude'], row['Longitude'],y_row['latitude'],y_row['longitude'])
          if distance < min_distance:
              min_distance = distance
              closest_rwi = y_row['rwi']

      print(f"Processed row {row.name + 1} - Closest rwi: {closest_rwi}")
      return closest_rwi

  x_data['Closest_rwi'] = x_data.apply(find_closest_rwi, axis=1)

  num_assigned_rwi = x_data['Closest_rwi'].count()

  output_path = '/content/csv file with labels.csv'
  x_data.to_csv(output_path, index=False)

  print(f"CSV file saved to: {output_path}")
  print(f"Number of closest RWIs assigned: {num_assigned_rwi}")

Processed row 1 - Closest rwi: 0.512
Processed row 2 - Closest rwi: 0.512
Processed row 3 - Closest rwi: 0.512
Processed row 4 - Closest rwi: 0.512
Processed row 5 - Closest rwi: 0.512
Processed row 6 - Closest rwi: 0.512
Processed row 7 - Closest rwi: 0.512
Processed row 8 - Closest rwi: 0.512
Processed row 9 - Closest rwi: 0.512
Processed row 10 - Closest rwi: 0.129
Processed row 11 - Closest rwi: 0.6
Processed row 12 - Closest rwi: 0.6
Processed row 13 - Closest rwi: 0.129
Processed row 14 - Closest rwi: 0.6
Processed row 15 - Closest rwi: 0.6
Processed row 16 - Closest rwi: 0.129
Processed row 17 - Closest rwi: 0.6
Processed row 18 - Closest rwi: 0.6
Processed row 19 - Closest rwi: 0.938
Processed row 20 - Closest rwi: 0.938
Processed row 21 - Closest rwi: 0.349
Processed row 22 - Closest rwi: 0.938
Processed row 23 - Closest rwi: 0.938
Processed row 24 - Closest rwi: 0.349
Processed row 25 - Closest rwi: 0.938
Processed row 26 - Closest rwi: 0.938
Processed row 27 - Closest rwi: 0